# Attentional Networks in Computer Vision
Prepared by Comp411 Teaching Unit (TA Can Küçüksözen) in the context of Computer Vision with Deep Learning Course. Do not hesitate to ask in case you have any questions, contact me at: ckucuksozen19@ku.edu.tr

Up until this point, we have worked with deep fully-connected networks, convolutional networks and recurrent networks using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, on the other hand, most successful image processing methods use convolutional networks. However recent state-of-the-art results on computer vision realm are acquired using Attentional layers and Transformer architectures.

First you will implement several layer types that are used in fully attentional networks. You will then use these layers to train an Attentional Image Classification network, specifically a smaller version of Vision Transformer (VIT) on the CIFAR-10 dataset. The original paper can be accessed via the following link: https://arxiv.org/pdf/2010.11929.pdf

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp411/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp411/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [3]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


# Part II. Barebones Transformers: Self-Attentional Layer

Here you will complete the implementation of the Pytorch nn.module `SelfAttention`, which will perform the forward pass of a self-attentional layer. Our implementation of the SelfAttentional layer will include three distinct fully connected layers which will be responsible of:

1. A fully connected layer, `W_Q`, which will be used to project our input into `queries`
2. A fully connected layer, `W_K`, which will be used to project our input into `keys`
3. A fully connected layer, `W_V`, which will be used to project our input into `values`

After defining such three fully connected layers, and obtain our `queries, keys, and values` variables at the beginning of our forward pass, the following operations should be carried out in order to complete the attentional layer implementation.

1. Seperate each of `query, key, and value` projections into their respective heads. In other words, split the feature vector dimension of each matrix into necessarry number of chunks.

2. Compute the `Attention Scores` between each pair of sequence elements via conducting a scaled dot product operation between every pair of `queries` and `keys`. Note that `Attention Scores` matrix should have the size of `[# of queries , # of keys]`

3. Calculate the `Attention Weights` of each query by applying the non-linear `Softmax` normalization accross the `keys` dimension of the `Attention Scores` matrix.

4. Obtain the output combination of `values` by matrix multiplying `Attention Weights` with `values`

5. Reassemble heads into one flat vector and return the output.

**HINT**: For a more detailed explanation of the self attentional layer, examine the Appendix A of the original ViT manuscript here:  https://arxiv.org/pdf/2010.11929.pdf 

In [4]:
class SelfAttention(nn.Module):
    
    def __init__(self, input_dims, head_dims=128, num_heads=2,  bias=False):
        super(SelfAttention, self).__init__()
        
        ## initialize module's instance variables
        self.input_dims = input_dims
        self.head_dims = head_dims
        self.num_heads = num_heads
        self.proj_dims = head_dims * num_heads
        
        ## Declare module's parameters
        self.W_Q = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_K = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_V = nn.Linear(input_dims, self.proj_dims,bias=bias)
        
        self.W_O = nn.Linear(self.proj_dims,self.proj_dims,bias=bias)

        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        ## Input of shape, [B, N, D] where:
        ## - B denotes the batch size
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D corresponds to model dimensionality
        b,n,d = x.shape
        
        ## Construct queries,keys,values
        q_ = self.W_Q(x)
        k_ = self.W_K(x)
        v_ = self.W_V(x)
        
        ## Seperate q,k,v into their corresponding heads,
        ## After this operation each q,k,v will have the shape: [B,H,N,D//H] where
        ## - B denotes the batch size
        ## - H denotes number of heads
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D//H corresponds to per head dimensionality
        q, k, v = map(lambda z: torch.reshape(z, (b,n,self.num_heads,self.head_dims)).permute(0,2,1,3), [q_,k_,v_])
       
        attn_out = None
        
        #########################################################################################
        # TODO: Complete the forward pass of the SelfAttention layer, follow the comments below #
        #########################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ## Compute attention logits. Note that this operation is implemented as a
        ## batched matrix multiplication between q and k, the output is scaled by 1/(D//H)^(1/2)
        ## inputs are queries and keys that are both of size [B,H,N,D//H]
        ## Output Attention logits should have the size: [B,H,N,N]
        
        h = q.shape[1]
        #print(f"B: {b}, H: {h}, N: {n}, D: {d}")
        attention_logits = (q @ k.transpose(-2, -1)) * (1/np.sqrt(b//h))       
        #print(f"attention_logits: {attention_logits.shape}")
        
        ## Compute attention Weights. Recall that this operation is conducted as a
        ## Softmax Normalization across the keys dimension. 
        ## Hint: You can apply the Softmax operation across the final dimension
        
        normalized_attention_logits = F.softmax(attention_logits, dim=-1)
        #print(f"normalized_attention_logits: {normalized_attention_logits.shape}")
        
        ## Compute attention output values. Bear in mind that this operation is applied as a 
        ## batched matrix multiplication between the Attention Weights matrix and 
        ## the values tensor. After computing output values, the output should be reshaped
        ## Inputs are Attention Weights with size [B, H, N, N], values with size [B, H, N, D//H]
        ## Output should be of size [B, N, D]
        ## Hint: you should use torch.matmul, torch.permute, torch.reshape in that order
        ##       (or any other equivalent torch operations)
        
        #print(f"V: {v.shape}")
        attention_output_logits = normalized_attention_logits @ v
        #print(f"attention_output_logits: {attention_output_logits.shape}")
        attention_output_logits = attention_output_logits.permute(0, 2, 1, 3).reshape(b, n, -1)
        #print(f"attention_output_logits: {attention_output_logits.shape}")
        
        ## Compute output feature map. This operation is just passing the concatenated attention 
        ## output that we have just obtained through a final projection layer W_O.
        ## Both the input and the output should be of size [B, N, D]
        
        attn_out = self.W_O(attention_output_logits)
        #print(f"attn_out: {attn_out.shape}")
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             
        ################################################################################
        
        
        
    
        return attn_out

After defining the forward pass of the Self-Attentional Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [5]:
def test_self_attn_layer():
    x = torch.zeros((64, 16, 32), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 32
    layer = SelfAttention(32,64,4)
    out = layer(x)
    print(out.size())  # you should see [64,16,256]
test_self_attn_layer()

torch.Size([64, 16, 256])


# Part III. Barebones Transformers: Transformer Encoder Block

Here you will complete the implementation of the Pytorch nn.module `TransformerBlock`, which will perform the forward pass of a Transfomer Encoder Block. You can refer to Figure 1 of the original manuscript of ViT from this link: https://arxiv.org/pdf/2010.11929.pdf in order to get yourself familiar with the architecture.



In [6]:
## Implementation of a two layer GELU activated Fully Connected Network is provided for you below:

class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, bias=True):
        super().__init__()
        
        self.fc_1 = nn.Linear(input_dims, hidden_dims, bias=bias)
        self.fc_2 = nn.Linear(hidden_dims, output_dims, bias=bias)
        
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)
        
    def forward(self, x):
        o = F.gelu(self.fc_1(x))
        o = self.fc_2(o)
        return o

In [7]:
## Build from scratch a TransformerBlock Module. Note that the architecture of this
## module follows a simple computational pipeline:
## input --> layernorm --> SelfAttention --> skip connection 
##       --> layernorm --> MLP ---> skip connection ---> output
## Note that the TransformerBlock module works on a single hidden dimension hidden_dims,
## in order to faciliate skip connections with ease. Be careful about the input arguments
## to the SelfAttention block.


class TransformerBlock(nn.Module):
    def __init__(self, hidden_dims, num_heads=4, bias=False):
        super(TransformerBlock, self).__init__()
        
###############################################################
# TODO: Complete the consturctor of  TransformerBlock module  #
###############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        self.attention_norm_layer = nn.LayerNorm(hidden_dims)
        self.multihead_attention  = SelfAttention(hidden_dims, head_dims=32, num_heads=num_heads,  bias=False)
        self.mlp_norm_layer       = nn.LayerNorm(hidden_dims)
        self.mlp                  = MLP(hidden_dims, hidden_dims*4, hidden_dims, bias=True)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################
        
    def forward(self, x):
        
##############################################################
# TODO: Complete the forward of TransformerBlock module      #
##############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        res1 = x
        #print(f"res1: {res1.shape}")
        x = self.attention_norm_layer(x)
        #print(f"attention_norm_layer: {x.shape}")
        x = self.multihead_attention(x)
        #print(f"multihead_attention: {x.shape}")
        x = x + res1
        #print(f"x: {x.shape}")
        
        res2 = x
        #print(f"res2: {res2.shape}")
        x = self.mlp_norm_layer(x)
        #print(f"mlp_norm_layer: {x.shape}")
        x = self.mlp(x)
        #print(f"mlp: {x.shape}")
        x = x + res2
        #print(f"out: {x.shape}")
        return x
 # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################

After defining the forward pass of the Transformer Block Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [8]:
def test_transfomerblock_layer():
    x = torch.zeros((64, 16, 128), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 128
    layer = TransformerBlock(128,4) # hidden dims size 128, heads size 4
    out = layer(x)
    print(out.size())  # you should see [64,16,128]
test_transfomerblock_layer()

torch.Size([64, 16, 128])


# Part IV The Vision Transformer (ViT)

The final implementation for the Pytorch nn.module `ViT` is given to you below, which will perform the forward pass of the Vision Transformer. Study it and get yourself familiar with the API.


In [9]:
class ViT(nn.Module):
    def __init__(self, hidden_dims, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4, bias=False):
        super(ViT, self).__init__()
                
        ## initialize module's instance variables
        self.hidden_dims = hidden_dims
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_trans_layers = num_trans_layers
        self.num_heads = num_heads
        self.image_k = image_k
        self.patch_k = patch_k
        
        self.image_height = self.image_width = image_k
        self.patch_height = self.patch_width = patch_k
        
        assert self.image_height % self.patch_height == 0 and self.image_width % self.patch_width == 0,\
                'Image size must be divisible by the patch size.'

        self.num_patches = (self.image_height // self.patch_height) * (self.image_width // self.patch_width)
        self.patch_flat_len = self.patch_height * self.patch_width
        
        ## Declare module's parameters
        
        ## ViT's flattened patch embedding projection:
        self.linear_embed = nn.Linear(self.input_dims*self.patch_flat_len, self.hidden_dims)
        
        ## Learnable positional embeddings, an embedding is learned for each patch location and the class token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, self.hidden_dims))
        
        ## Learnable classt token and its index among attention sequence elements.
        self.cls_token = nn.Parameter(torch.randn(1,1,self.hidden_dims))
        self.cls_index = torch.LongTensor([0])
        
        ## Declare cascaded Transformer blocks:
        transformer_encoder_list = []
        for _ in range(self.num_trans_layers):
            transformer_encoder_list.append(TransformerBlock(self.hidden_dims, self.num_heads, bias))
        self.transformer_encoder = nn.Sequential(*transformer_encoder_list)
        
        ## Declare the output mlp:
        self.out_mlp = MLP(self.hidden_dims, self.hidden_dims, self.output_dims)
         
    def unfold(self, x, f = 7, st = 4, p = 0):
        ## Create sliding window pathes using nn.Functional.unfold
        ## Input dimensions: [B,D,H,W] where
        ## --B : input batch size
        ## --D : input channels
        ## --H, W: input height and width
        ## Output dimensions: [B,N,H*W,D]
        ## --N : number of patches, decided according to sliding window kernel size (f),
        ##      sliding window stride and padding.
        b,d,h,w = x.shape
        x_unf = F.unfold(x, (f,f), stride=st, padding=p)    
        x_unf = torch.reshape(x_unf.permute(0,2,1), (b,-1,d,f*f)).transpose(-1,-2)
        n = x_unf.size(1)
        return x_unf,n
    
    def forward(self, x):
        b = x.size(0)
        ## create sliding window patches from the input image
        x_patches,n = self.unfold(x, self.patch_height, self.patch_height, 0)
        ## flatten each patch into a 1d vector: i.e. 3x4x4 image patch turned into 1x1x48
        x_patch_flat = torch.reshape(x_patches, (b,n,-1))
        ## linearly embed each flattened patch
        x_embed = self.linear_embed(x_patch_flat)
        
        ## retrieve class token 
        cls_tokens = self.cls_token.repeat(b,1,1)
        ## concatanate class token to input patches
        xcls_embed = torch.cat([cls_tokens, x_embed], dim=-2)
        
        ## add positional embedding to input patches + class token 
        xcls_pos_embed = xcls_embed + self.pos_embedding
        
        ## pass through the transformer encoder
        trans_out = self.transformer_encoder(xcls_pos_embed)
        
        ## select the class token 
        out_cls_token = torch.index_select(trans_out, -2, self.cls_index.to(trans_out.device))
        
        ## create output
        out = self.out_mlp(out_cls_token)
        
        return out.squeeze(-2)

After defining the forward pass of the ViT above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [10]:
def test_vit():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size 3,32,32
    model = ViT(hidden_dims=128, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4)
    out = model(x)
    print(out.size())  # you should see [64,10]
test_vit()

torch.Size([64, 10])


# Part V. Train the ViT

### Check Accuracy
Given any minibatch of input data and desired targets, we can check the classification accuracy of a neural network. 

The check_batch_accuracy function is provided for you below:

In [11]:
def check_batch_accuracy(out, target,eps=1e-7):
    b, c = out.shape
    with torch.no_grad():
        _, pred = out.max(-1) 
        correct = np.sum(np.equal(pred.cpu().numpy(), target.cpu().numpy()))
    return correct, np.float(correct) / (b)

### Training Loop
As we have already seen in the Second Assignment, in our PyTorch based training loops, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [12]:
def train(network, optimizer, trainloader):
    """
    Train a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - trainloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall training accuracy for the epoch
    """
    print('\nEpoch: %d' % epoch)
    network.train()  # put model to training mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = Variable(inputs.to(device)), targets.to(device)  # move to device, e.g. GPU
            
        outputs = network(inputs)
        loss =  F.cross_entropy(outputs, targets)
            
        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad() 

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()
            
        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()
            
        loss = loss.detach()
        train_loss += loss.item()
        correct_p, _ = check_batch_accuracy(outputs, targets) 
        correct += correct_p
        total += targets.size(0)

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    return 100.*correct/total

### Evaluation Loop
We have also prepared a Evaluation loop in order to determine our networks capabilities in terms of classification accuracy on a given dataset, either the training, or the validation split

In [13]:
def evaluate(network, evalloader):
    """
    Evaluate a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - evalloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall evaluation accuracy for the epoch
    """
    network.eval() # put model to evaluation mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    eval_loss = 0
    correct = 0
    total = 0
    print('\n---- Evaluation in process ----')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(evalloader):
            inputs, targets = inputs.to(device), targets.to(device) # move to device, e.g. GPU
            outputs = network(inputs)
            loss = F.cross_entropy(outputs, targets)
            
            eval_loss += loss.item()
            correct_p, _ = check_batch_accuracy(outputs, targets)
            correct += correct_p
            total += targets.size(0)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (eval_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total

### Overfit a ViT
Now we are ready to run the training loop. A nice trick is to train your model with just a few training samples in order to see if your implementation is actually bug free. 

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `ViT`. 

You also need to define an optimizer that tracks all the learnable parameters inside `ViT`. We prefer to use `Adam` optimizer for this part.

You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

In [14]:
sample_idx_tr = torch.randperm(len(cifar10_train))[:100]
sample_idx_val = torch.randperm(len(cifar10_train))[-100:]

trainset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_tr)
valset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_val)

print("For overfitting experiments, the subset of the dataset that is used has {} sample images".format(len(trainset_sub)))

batch_size_sub = 25
trainloader_sub = torch.utils.data.DataLoader(trainset_sub, batch_size=batch_size_sub, shuffle=True)
valloader_sub = torch.utils.data.DataLoader(valset_sub, batch_size=batch_size_sub, shuffle=False)

print('==> Data ready, batchsize = {}'.format(batch_size_sub))

For overfitting experiments, the subset of the dataset that is used has 100 sample images
==> Data ready, batchsize = 25


In [15]:
learning_rate = 0.001
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

model = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims=hidden_dims, 
            input_dims=input_dims, 
            output_dims=output_dims, 
            num_trans_layers=num_trans_layers, 
            num_heads=num_heads, 
            image_k=image_k, 
            patch_k=patch_k, 
            bias=False).to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=0)
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
eval_accs=[]
for epoch in range(15):
    tr_acc = train(network, optimizer, trainloader_sub)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    eval_acc = evaluate(network, valloader_sub)
    print('Evaluation of Epoch {} is completed, Validation accuracy for this epoch is {}'\
              .format(epoch, eval_acc))  
    tr_accs.append(tr_acc)
    eval_accs.append(eval_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final val set accuracy is {}".format(eval_accs[-1]))


Epoch: 0
Loss: 2.960 | Acc: 8.000% (2/25)
Loss: 4.362 | Acc: 6.000% (3/50)
Loss: 4.136 | Acc: 10.667% (8/75)
Loss: 4.041 | Acc: 10.000% (10/100)
Epoch 0 of training is completed, Training accuracy for this epoch is 10.0

---- Evaluation in process ----
Loss: 4.693 | Acc: 8.000% (2/25)
Loss: 4.698 | Acc: 10.000% (5/50)
Loss: 4.532 | Acc: 13.333% (10/75)
Loss: 4.712 | Acc: 12.000% (12/100)
Evaluation of Epoch 0 is completed, Validation accuracy for this epoch is 12.0

Epoch: 1
Loss: 4.411 | Acc: 8.000% (2/25)
Loss: 3.688 | Acc: 12.000% (6/50)
Loss: 3.299 | Acc: 10.667% (8/75)
Loss: 3.188 | Acc: 11.000% (11/100)
Epoch 1 of training is completed, Training accuracy for this epoch is 11.0

---- Evaluation in process ----
Loss: 2.376 | Acc: 28.000% (7/25)
Loss: 2.488 | Acc: 22.000% (11/50)
Loss: 2.653 | Acc: 20.000% (15/75)
Loss: 2.662 | Acc: 17.000% (17/100)
Evaluation of Epoch 1 is completed, Validation accuracy for this epoch is 17.0

Epoch: 2
Loss: 2.266 | Acc: 24.000% (6/25)
Loss: 2.228

## Train the net
By training the four-layer ViT network for three epochs, with untuned hyperparameters that are initialized as below,  you should achieve greater than 50% accuracy both on the training set and the test set:

In [16]:
learning_rate = 0.001
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
network = ViT(hidden_dims=hidden_dims, 
            input_dims=input_dims, 
            output_dims=output_dims, 
            num_trans_layers=num_trans_layers, 
            num_heads=num_heads, 
            image_k=image_k, 
            patch_k=patch_k, 
            bias=False).to(device)

optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=0)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
test_accs=[]
for epoch in range(3):
    tr_acc = train(network, optimizer, loader_train)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    test_acc = evaluate(network, loader_test)
    print('Evaluation of Epoch {} is completed, Test accuracy for this epoch is {}'\
              .format(epoch, test_acc))  
    
    tr_accs.append(tr_acc)
    test_accs.append(test_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final test set accuracy is {}".format(test_accs[-1]))


Epoch: 0
Loss: 4.624 | Acc: 3.125% (2/64)
Loss: 4.849 | Acc: 4.688% (6/128)
Loss: 4.562 | Acc: 7.292% (14/192)
Loss: 4.581 | Acc: 7.812% (20/256)
Loss: 4.197 | Acc: 9.375% (30/320)
Loss: 4.044 | Acc: 9.115% (35/384)
Loss: 3.892 | Acc: 9.821% (44/448)
Loss: 3.749 | Acc: 10.547% (54/512)
Loss: 3.625 | Acc: 11.111% (64/576)
Loss: 3.514 | Acc: 12.031% (77/640)
Loss: 3.397 | Acc: 13.210% (93/704)
Loss: 3.307 | Acc: 13.932% (107/768)
Loss: 3.252 | Acc: 14.423% (120/832)
Loss: 3.180 | Acc: 14.732% (132/896)
Loss: 3.104 | Acc: 15.208% (146/960)
Loss: 3.060 | Acc: 15.039% (154/1024)
Loss: 3.020 | Acc: 15.257% (166/1088)
Loss: 2.977 | Acc: 15.538% (179/1152)
Loss: 2.954 | Acc: 15.872% (193/1216)
Loss: 2.924 | Acc: 15.938% (204/1280)
Loss: 2.895 | Acc: 16.146% (217/1344)
Loss: 2.858 | Acc: 16.832% (237/1408)
Loss: 2.837 | Acc: 16.984% (250/1472)
Loss: 2.811 | Acc: 16.992% (261/1536)
Loss: 2.786 | Acc: 17.312% (277/1600)
Loss: 2.766 | Acc: 17.728% (295/1664)
Loss: 2.749 | Acc: 17.535% (303/1728)


Loss: 2.002 | Acc: 29.572% (4107/13888)
Loss: 2.001 | Acc: 29.652% (4137/13952)
Loss: 1.998 | Acc: 29.688% (4161/14016)
Loss: 1.997 | Acc: 29.709% (4183/14080)
Loss: 1.996 | Acc: 29.695% (4200/14144)
Loss: 1.995 | Acc: 29.744% (4226/14208)
Loss: 1.993 | Acc: 29.793% (4252/14272)
Loss: 1.992 | Acc: 29.792% (4271/14336)
Loss: 1.990 | Acc: 29.826% (4295/14400)
Loss: 1.989 | Acc: 29.833% (4315/14464)
Loss: 1.987 | Acc: 29.901% (4344/14528)
Loss: 1.986 | Acc: 29.941% (4369/14592)
Loss: 1.985 | Acc: 29.960% (4391/14656)
Loss: 1.984 | Acc: 29.980% (4413/14720)
Loss: 1.982 | Acc: 29.999% (4435/14784)
Loss: 1.982 | Acc: 30.011% (4456/14848)
Loss: 1.981 | Acc: 30.036% (4479/14912)
Loss: 1.979 | Acc: 30.088% (4506/14976)
Loss: 1.977 | Acc: 30.160% (4536/15040)
Loss: 1.977 | Acc: 30.184% (4559/15104)
Loss: 1.976 | Acc: 30.228% (4585/15168)
Loss: 1.975 | Acc: 30.219% (4603/15232)
Loss: 1.974 | Acc: 30.263% (4629/15296)
Loss: 1.973 | Acc: 30.286% (4652/15360)
Loss: 1.973 | Acc: 30.316% (4676/15424)


Loss: 1.821 | Acc: 35.014% (9479/27072)
Loss: 1.820 | Acc: 35.027% (9505/27136)
Loss: 1.820 | Acc: 35.029% (9528/27200)
Loss: 1.819 | Acc: 35.050% (9556/27264)
Loss: 1.819 | Acc: 35.067% (9583/27328)
Loss: 1.819 | Acc: 35.072% (9607/27392)
Loss: 1.818 | Acc: 35.111% (9640/27456)
Loss: 1.817 | Acc: 35.131% (9668/27520)
Loss: 1.816 | Acc: 35.154% (9697/27584)
Loss: 1.816 | Acc: 35.167% (9723/27648)
Loss: 1.815 | Acc: 35.191% (9752/27712)
Loss: 1.814 | Acc: 35.232% (9786/27776)
Loss: 1.814 | Acc: 35.230% (9808/27840)
Loss: 1.814 | Acc: 35.257% (9838/27904)
Loss: 1.813 | Acc: 35.269% (9864/27968)
Loss: 1.812 | Acc: 35.281% (9890/28032)
Loss: 1.812 | Acc: 35.311% (9921/28096)
Loss: 1.811 | Acc: 35.327% (9948/28160)
Loss: 1.810 | Acc: 35.349% (9977/28224)
Loss: 1.810 | Acc: 35.365% (10004/28288)
Loss: 1.809 | Acc: 35.384% (10032/28352)
Loss: 1.809 | Acc: 35.385% (10055/28416)
Loss: 1.809 | Acc: 35.390% (10079/28480)
Loss: 1.808 | Acc: 35.440% (10116/28544)
Loss: 1.807 | Acc: 35.487% (10152/2

Loss: 1.729 | Acc: 37.996% (15247/40128)
Loss: 1.728 | Acc: 38.005% (15275/40192)
Loss: 1.728 | Acc: 38.022% (15306/40256)
Loss: 1.728 | Acc: 38.041% (15338/40320)
Loss: 1.727 | Acc: 38.052% (15367/40384)
Loss: 1.727 | Acc: 38.056% (15393/40448)
Loss: 1.727 | Acc: 38.078% (15426/40512)
Loss: 1.726 | Acc: 38.099% (15459/40576)
Loss: 1.726 | Acc: 38.100% (15484/40640)
Loss: 1.726 | Acc: 38.099% (15508/40704)
Loss: 1.725 | Acc: 38.113% (15538/40768)
Loss: 1.724 | Acc: 38.137% (15572/40832)
Loss: 1.724 | Acc: 38.153% (15603/40896)
Loss: 1.723 | Acc: 38.171% (15635/40960)
Loss: 1.723 | Acc: 38.180% (15663/41024)
Loss: 1.722 | Acc: 38.211% (15700/41088)
Loss: 1.721 | Acc: 38.234% (15734/41152)
Loss: 1.721 | Acc: 38.242% (15762/41216)
Loss: 1.721 | Acc: 38.266% (15796/41280)
Loss: 1.720 | Acc: 38.269% (15822/41344)
Loss: 1.720 | Acc: 38.273% (15848/41408)
Loss: 1.719 | Acc: 38.296% (15882/41472)
Loss: 1.719 | Acc: 38.302% (15909/41536)
Loss: 1.718 | Acc: 38.327% (15944/41600)
Loss: 1.718 | Ac

Loss: 1.485 | Acc: 45.879% (2349/5120)
Loss: 1.484 | Acc: 46.007% (2385/5184)
Loss: 1.485 | Acc: 45.979% (2413/5248)
Loss: 1.485 | Acc: 45.971% (2442/5312)
Loss: 1.488 | Acc: 45.889% (2467/5376)
Loss: 1.488 | Acc: 45.919% (2498/5440)
Loss: 1.488 | Acc: 45.858% (2524/5504)
Loss: 1.492 | Acc: 45.797% (2550/5568)
Loss: 1.494 | Acc: 45.774% (2578/5632)
Loss: 1.498 | Acc: 45.681% (2602/5696)
Loss: 1.497 | Acc: 45.712% (2633/5760)
Loss: 1.494 | Acc: 45.793% (2667/5824)
Loss: 1.496 | Acc: 45.720% (2692/5888)
Loss: 1.499 | Acc: 45.665% (2718/5952)
Loss: 1.500 | Acc: 45.612% (2744/6016)
Loss: 1.500 | Acc: 45.526% (2768/6080)
Loss: 1.499 | Acc: 45.589% (2801/6144)
Loss: 1.499 | Acc: 45.586% (2830/6208)
Loss: 1.501 | Acc: 45.456% (2851/6272)
Loss: 1.500 | Acc: 45.533% (2885/6336)
Loss: 1.501 | Acc: 45.484% (2911/6400)
Loss: 1.502 | Acc: 45.421% (2936/6464)
Loss: 1.502 | Acc: 45.450% (2967/6528)
Loss: 1.504 | Acc: 45.373% (2991/6592)
Loss: 1.503 | Acc: 45.418% (3023/6656)
Loss: 1.503 | Acc: 45.327

...

Loss: 1.299 | Acc: 53.869% (1448/2688)
Loss: 1.301 | Acc: 53.743% (1479/2752)
Loss: 1.299 | Acc: 53.800% (1515/2816)
Loss: 1.298 | Acc: 53.681% (1546/2880)
Loss: 1.297 | Acc: 53.635% (1579/2944)
Loss: 1.296 | Acc: 53.690% (1615/3008)
Loss: 1.296 | Acc: 53.646% (1648/3072)
Loss: 1.294 | Acc: 53.444% (1676/3136)
Loss: 1.293 | Acc: 53.438% (1710/3200)
Loss: 1.293 | Acc: 53.493% (1746/3264)
Loss: 1.289 | Acc: 53.636% (1785/3328)
Loss: 1.289 | Acc: 53.538% (1816/3392)
Loss: 1.288 | Acc: 53.704% (1856/3456)
Loss: 1.289 | Acc: 53.636% (1888/3520)
Loss: 1.288 | Acc: 53.627% (1922/3584)
Loss: 1.288 | Acc: 53.564% (1954/3648)
Loss: 1.285 | Acc: 53.691% (1993/3712)
Loss: 1.282 | Acc: 53.787% (2031/3776)
Loss: 1.285 | Acc: 53.750% (2064/3840)
Loss: 1.283 | Acc: 53.817% (2101/3904)
Loss: 1.282 | Acc: 53.881% (2138/3968)
Loss: 1.278 | Acc: 53.943% (2175/4032)
Loss: 1.274 | Acc: 54.077% (2215/4096)
Loss: 1.274 | Acc: 54.014% (2247/4160)
Loss: 1.273 | Acc: 54.096% (2285/4224)
Loss: 1.274 | Acc: 54.104

Loss: 1.253 | Acc: 54.755% (8866/16192)
Loss: 1.253 | Acc: 54.774% (8904/16256)
Loss: 1.253 | Acc: 54.792% (8942/16320)
Loss: 1.254 | Acc: 54.803% (8979/16384)
Loss: 1.253 | Acc: 54.797% (9013/16448)
Loss: 1.253 | Acc: 54.803% (9049/16512)
Loss: 1.253 | Acc: 54.814% (9086/16576)
Loss: 1.253 | Acc: 54.856% (9128/16640)
Loss: 1.252 | Acc: 54.873% (9166/16704)
Loss: 1.252 | Acc: 54.896% (9205/16768)
Loss: 1.252 | Acc: 54.907% (9242/16832)
Loss: 1.252 | Acc: 54.906% (9277/16896)
Loss: 1.252 | Acc: 54.935% (9317/16960)
Loss: 1.253 | Acc: 54.940% (9353/17024)
Loss: 1.252 | Acc: 54.933% (9387/17088)
Loss: 1.253 | Acc: 54.886% (9414/17152)
Loss: 1.253 | Acc: 54.885% (9449/17216)
Loss: 1.253 | Acc: 54.890% (9485/17280)
Loss: 1.252 | Acc: 54.912% (9524/17344)
Loss: 1.252 | Acc: 54.940% (9564/17408)
Loss: 1.252 | Acc: 54.945% (9600/17472)
Loss: 1.252 | Acc: 54.933% (9633/17536)
Loss: 1.252 | Acc: 54.909% (9664/17600)
Loss: 1.252 | Acc: 54.869% (9692/17664)
Loss: 1.252 | Acc: 54.868% (9727/17728)


Loss: 1.256 | Acc: 54.647% (16088/29440)
Loss: 1.255 | Acc: 54.643% (16122/29504)
Loss: 1.255 | Acc: 54.660% (16162/29568)
Loss: 1.255 | Acc: 54.667% (16199/29632)
Loss: 1.255 | Acc: 54.677% (16237/29696)
Loss: 1.255 | Acc: 54.657% (16266/29760)
Loss: 1.255 | Acc: 54.654% (16300/29824)
Loss: 1.255 | Acc: 54.661% (16337/29888)
Loss: 1.255 | Acc: 54.677% (16377/29952)
Loss: 1.255 | Acc: 54.674% (16411/30016)
Loss: 1.254 | Acc: 54.694% (16452/30080)
Loss: 1.254 | Acc: 54.684% (16484/30144)
Loss: 1.254 | Acc: 54.691% (16521/30208)
Loss: 1.254 | Acc: 54.701% (16559/30272)
Loss: 1.254 | Acc: 54.701% (16594/30336)
Loss: 1.254 | Acc: 54.720% (16635/30400)
Loss: 1.254 | Acc: 54.727% (16672/30464)
Loss: 1.254 | Acc: 54.730% (16708/30528)
Loss: 1.253 | Acc: 54.743% (16747/30592)
Loss: 1.253 | Acc: 54.749% (16784/30656)
Loss: 1.253 | Acc: 54.730% (16813/30720)
Loss: 1.253 | Acc: 54.726% (16847/30784)
Loss: 1.253 | Acc: 54.733% (16884/30848)
Loss: 1.253 | Acc: 54.730% (16918/30912)
Loss: 1.253 | Ac

Loss: 1.250 | Acc: 54.810% (23327/42560)
Loss: 1.250 | Acc: 54.814% (23364/42624)
Loss: 1.250 | Acc: 54.823% (23403/42688)
Loss: 1.249 | Acc: 54.835% (23443/42752)
Loss: 1.249 | Acc: 54.832% (23477/42816)
Loss: 1.249 | Acc: 54.839% (23515/42880)
Loss: 1.249 | Acc: 54.851% (23555/42944)
Loss: 1.249 | Acc: 54.855% (23592/43008)
Loss: 1.249 | Acc: 54.850% (23625/43072)
Loss: 1.249 | Acc: 54.841% (23656/43136)
Loss: 1.249 | Acc: 54.850% (23695/43200)
Loss: 1.249 | Acc: 54.847% (23729/43264)
Loss: 1.250 | Acc: 54.838% (23760/43328)
Loss: 1.250 | Acc: 54.844% (23798/43392)
Loss: 1.250 | Acc: 54.846% (23834/43456)
Loss: 1.250 | Acc: 54.844% (23868/43520)
Loss: 1.250 | Acc: 54.850% (23906/43584)
Loss: 1.249 | Acc: 54.859% (23945/43648)
Loss: 1.249 | Acc: 54.866% (23983/43712)
Loss: 1.249 | Acc: 54.868% (24019/43776)
Loss: 1.249 | Acc: 54.868% (24054/43840)
Loss: 1.249 | Acc: 54.870% (24090/43904)
Loss: 1.249 | Acc: 54.874% (24127/43968)
Loss: 1.249 | Acc: 54.885% (24167/44032)
Loss: 1.249 | Ac

Loss: 1.240 | Acc: 54.406% (3865/7104)
Loss: 1.240 | Acc: 54.422% (3901/7168)
Loss: 1.240 | Acc: 54.397% (3934/7232)
Loss: 1.238 | Acc: 54.468% (3974/7296)
Loss: 1.236 | Acc: 54.552% (4015/7360)
Loss: 1.235 | Acc: 54.512% (4047/7424)
Loss: 1.234 | Acc: 54.487% (4080/7488)
Loss: 1.234 | Acc: 54.529% (4118/7552)
Loss: 1.235 | Acc: 54.451% (4147/7616)
Loss: 1.234 | Acc: 54.479% (4184/7680)
Loss: 1.233 | Acc: 54.571% (4226/7744)
Loss: 1.233 | Acc: 54.611% (4264/7808)
Loss: 1.235 | Acc: 54.548% (4294/7872)
Loss: 1.234 | Acc: 54.574% (4331/7936)
Loss: 1.235 | Acc: 54.625% (4370/8000)
Loss: 1.235 | Acc: 54.601% (4403/8064)
Loss: 1.235 | Acc: 54.577% (4436/8128)
Loss: 1.235 | Acc: 54.614% (4474/8192)
Loss: 1.235 | Acc: 54.603% (4508/8256)
Loss: 1.237 | Acc: 54.555% (4539/8320)
Loss: 1.237 | Acc: 54.568% (4575/8384)
Loss: 1.237 | Acc: 54.534% (4607/8448)
Loss: 1.238 | Acc: 54.500% (4639/8512)
Loss: 1.239 | Acc: 54.478% (4672/8576)
Loss: 1.238 | Acc: 54.479% (4707/8640)
Loss: 1.239 | Acc: 54.412